In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from numpy.lib.stride_tricks import sliding_window_view
import time
from src.mplstyle import set as set_mplstyle
set_mplstyle("Fira Sans")

HPD_PATH = Path("/mnt/d/climate_data/hpd/data/")
GHCND_PATH = Path("/mnt/d/climate_data/ghcnd/data/")
DATA_DOC_PATH = Path("./data/dataset_docs/")
OUTPUT_PATH = Path("/mnt/d/climate_data/HPD_CONUS_OEVENTS/")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

# Identify potential stations

Potential stations are stations where both PRCP and TMEAN observations are available over a POR of at least 30 Years. The ordinary events at each stations in the potential station lists are first computed. The joint data requirement is applied once the ordinary events are computed and paired with temperature observation

In [2]:
def compute_PT_overlapping_years(df):
    # Extract start and end years
    df['P_start'] = df['POR_Date_Range_P'].str.slice(0, 4).astype(int)
    df['P_end'] = df['POR_Date_Range_P'].str.slice(9, 13).astype(int)
    df['T_start'] = df['POR_Date_Range_T'].str.slice(0, 4).astype(int)
    df['T_end'] = df['POR_Date_Range_T'].str.slice(9, 13).astype(int)

    # Compute overlap
    df['overlap_start'] = df[['P_start', 'T_start']].max(axis=1)
    df['overlap_end'] = df[['P_end', 'T_end']].min(axis=1)

    # Count overlapping years
    df['overlap_years'] = (df['overlap_end'] - df['overlap_start'] + 1).clip(lower=0)

    # Drop intermediate columns if desired
    df = df.drop(columns=['P_start', 'P_end', 'T_start', 'T_end', 'overlap_start', 'overlap_end'])
    return df

In [3]:
# HPD Stations
hpd_stations = (
    pd.read_csv(DATA_DOC_PATH / "HPD_v02r02_stationinv_c20240502.csv")
    .drop(columns=['Last_Half_POR', 'PCT_Last_Half_Good', 'Last_Qtr_POR', 'PCT_Last_Qtr_Good', 'Sample_Interval (min)', 'WMO_ID'])
)
hpd_stations['PCT_POR_Good'] = hpd_stations['PCT_POR_Good'].str.rstrip('%').astype(float)
hpd_stations['Num_Years'] = (hpd_stations['POR_Date_Range'].str.slice(9, 13).astype(int) -
                              hpd_stations['POR_Date_Range'].str.slice(0, 4).astype(int))

ghcnd_stations = pd.read_csv(DATA_DOC_PATH / "GHCND_TMEAN_station_info.csv")
ghcnd_stations = ghcnd_stations[ghcnd_stations["Num_Years"] > 0]
pt_stations = hpd_stations.merge(ghcnd_stations.drop(columns=['LAT', 'LON', 'ELEV', 'STATE', 'NAME']), on='StnID', how='inner', suffixes=('_P', '_T'))
pt_stations = compute_PT_overlapping_years(pt_stations)
pt_stations

,StnID,Lat,Lon,Elev,State/Province,Name,UTC_Offset,POR_Date_Range_P,PCT_POR_Good_P,Num_Years_P,POR_Date_Range_T,Num_Years_T,PCT_POR_Good_T,overlap_years
0,USC00010063,34.2110,-87.1784,239.6,AL,ADDISON,-6,19480601-20240416,84.6,76,19380301-20240503,87.0,19.5,77
1,USC00010252,31.3071,-86.5226,76.2,AL,ANDALUSIA 3 W,-6,19800301-20180205,89.1,38,19121005-20180131,107.0,70.6,39
2,USC00010369,33.2941,-85.7788,311.5,AL,ASHLAND 3 ENE,-6,19480601-20130804,84.6,65,19400301-20130731,74.0,71.9,66
3,USC00010390,34.7752,-86.9508,210.0,AL,ATHENS,-6,19820601-20240304,84.5,42,19410520-20240504,84.0,38.4,43
4,USC00010402,31.1820,-87.4390,91.4,AL,ATMORE,-6,19480601-20240401,64.4,76,19400327-20240430,85.0,73.4,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,USC00518838,21.4992,-158.0111,306.6,HI,UPPER WAHIAWA 874.3,-10,20150110-20240422,80.1,9,19710401-20150430,45.0,29.1,1
1435,USC00518941,21.8967,-159.5569,65.5,HI,WAHIAWA 930,-10,19650301-20170410,87.5,52,19491001-20240329,76.0,8.0,53
1436,USC00518945,21.4967,-158.0497,260.3,HI,WAHIAWA DAM 863,-10,19650419-20240422,94.7,59,19400101-19780430,39.0,1.9,14
1437,USC00519195,21.5747,-158.1204,3.0,HI,WAIALUA 847,-10,19650301-20240311,89.3,59,19080101-20011130,94.0,67.3,37


In [4]:
potential_stations = pt_stations.query("24.5 < Lat < 49.5 and -125 < Lon < -66.75 and overlap_years >= 30").sort_values(['overlap_years'],ascending=False).reset_index(drop=True)
potential_stations

,StnID,Lat,Lon,Elev,State/Province,Name,UTC_Offset,POR_Date_Range_P,PCT_POR_Good_P,Num_Years_P,POR_Date_Range_T,Num_Years_T,PCT_POR_Good_T,overlap_years
0,USC00410569,28.9798,-95.9749,15.8,TX,BAY CITY WATERWORKS,-6,19400601-20240417,57.1,84,19091001-20240331,116.0,70.2,85
1,USC00241737,47.8205,-112.1921,1172.0,MT,CHOTEAU,-7,19400101-20240401,93.8,84,18930101-20240504,132.0,83.6,85
2,USW00093914,31.7831,-95.6039,141.7,TX,PALESTINE 2 NE,-6,19400201-20240213,86.7,84,19300101-20240324,95.0,92.2,85
3,USC00414670,30.4452,-99.8044,532.5,TX,JUNCTION 4SSW,-6,19400301-20240401,90.4,84,18970127-20240503,128.0,75.0,85
4,USC00414570,33.2543,-100.5731,612.6,TX,JAYTON,-6,19400501-20240405,90.4,84,19100601-20240503,115.0,51.4,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042,USC00170100,44.3738,-68.2591,143.3,ME,ACADIA NATIONAL PARK,-5,19820907-20120704,86.8,30,19820901-20140531,33.0,94.8,31
1043,USC00478316,42.9676,-88.5495,283.2,WI,SULLIVAN 3 SE - WFO MKX,-6,19940901-20240402,85.6,30,19950101-20240504,30.0,96.8,30
1044,USC00084095,25.5011,-80.5500,2.7,FL,HOMESTEAD GEN AVIATION AP,-5,19950701-20240402,84.0,29,19900601-20240504,35.0,88.2,30
1045,USC00033544,36.0700,-93.7522,543.5,AR,HUNTSVILLE 1 SSW,-6,19830501-20120925,90.1,29,19830501-20120924,30.0,97.2,30


In [5]:
potential_stations.to_csv(DATA_DOC_PATH / "potential_pt_stations.csv", index=False)

# Compute Ordinary Events

In [6]:
def load_hpd_data(stn_id):
    """
    Load HPD data into a Pandas DataFrame with datetime index, making sure all observation inside flagged accumulation period are removed
    """
    hourly_cols = [f"HR{i:02d}Val" for i in range(24)]
    MF_cols = [f"HR{i:02d}MF" for i in range(24)]
    cols =['DATE', *hourly_cols, *MF_cols]

    # Load and reindex data to identify missing days
    df = pd.read_csv(HPD_PATH / f"{stn_id}.csv", low_memory=False, na_values=[9999, -9999], usecols=cols, index_col='DATE', parse_dates=True).asfreq('D')

    prcp = df[hourly_cols].values.flatten() * 0.254 # Hundredth of inch --> mm
    flags = df[MF_cols].values.flatten()

    # # Remove all non-missing observation inside flagged accumulation period 
    #len_before = len(prcp[~np.isnan(prcp)])
    accum_mask = np.isin(flags, ['a', '.', 'A']).astype(bool)
    prcp[accum_mask] = np.nan
    #print(f"{len_before - len(prcp[~np.isnan(prcp)])} suspicious data removed")

    # Time index:
    repeated_dates = np.repeat(df.index, 24)
    hours = np.tile(np.arange(24), len(df))
    index = repeated_dates + pd.to_timedelta(hours, unit='h')
    return pd.DataFrame({'PRCP': prcp}, index=index)

def ordinary_events(data, timeindex, durations, delta_t, thresh=0.254, sep_len="24h"):
    """ 
    Split data into storms
    data: 1 for precipitation data and 0 for dry period
    Note: 0s are not removed at the begining and at the end if they are not longer than sep_len. Needs modification if interested in storm duration.
    Args:
    ----
    delta_t: Time intervals between samples (Pandas frequency format e.g. "24H")
    """

    
    data = np.array(data)

    if not isinstance(durations, list):
        durations = [durations]

    # Find where 0 runs start and their lengths
    is_zero = (data < thresh).astype(int)
    changes = np.diff(is_zero.astype(int), prepend=0, append=0)

    zero_starts = np.where(changes == 1)[0]
    zero_ends = np.where(changes == -1)[0]
    zero_lengths = zero_ends - zero_starts

    # Indices of zero runs that are at least sep_len long
    sep_len_su = int(pd.Timedelta(sep_len) / pd.Timedelta(delta_t)) # sep_len in sample units
    long_zero_spans = [(start, end) for start, end, length in zip(zero_starts, zero_ends, zero_lengths) if length >= sep_len_su]

    # Slice the data
    storms = []
    storms_time = []
    prev_end = 0
    for start, end in long_zero_spans:
        if prev_end < start:
            storms.append(data[prev_end:start])
            storms_time.append(timeindex[prev_end:start])
        prev_end = end
    if prev_end < len(data):
        storms.append(data[prev_end:])
        storms_time.append(timeindex[prev_end:])
    #return [arr for arr in storms if len(arr) > 0 and not np.isnan(arr).all()] # If not missing

    # Identify events
    ## Compute window size from duration
    window_sizes = [int(pd.Timedelta(d) / pd.Timedelta(delta_t)) for d in durations]
    res = []
    for ws in window_sizes:
        events = {'datetime': [], 'PRCP': []}
        for jj, storm in enumerate(storms):
            if len(storm) >= ws and not np.isnan(storm).all(): # If not missing
                if ws == 1:
                    max_idx = np.nanargmax(storm)
                    storm_max = storm[max_idx]
                    event_time = storms_time[jj][max_idx]
                else:
                    windows = sliding_window_view(storm, ws) # safer than np.lib.stride_tricks.as_strided
                    agg_values = np.nansum(windows, axis=1)
                    max_idx = agg_values.argmax()
                    storm_max = agg_values[max_idx]
                    event_time = storms_time[jj][max_idx + ws - 1]

                events["datetime"].append(event_time)
                events["PRCP"].append(storm_max)
            else:
                continue
            
        res.append(pd.DataFrame(events).set_index("datetime"))
    return res


def compute_ordinary_events_from_hpd(stations, durations):
    duration_dirs = {d: (OUTPUT_PATH / d) for d in durations}
    for path in duration_dirs.values():
        path.mkdir(parents=True, exist_ok=True)

    for stn_id in stations:
        prcp_df = load_hpd_data(stn_id)
        oes_list = ordinary_events(
            prcp_df["PRCP"].values,
            prcp_df.index.values,
            durations=durations,
            delta_t="1h",
            thresh=0.254,
            sep_len="24h",
        )
        for d, df in zip(durations, oes_list):
            df.to_csv(duration_dirs[d] / f"{stn_id}.csv")
    

In [7]:
stations = potential_stations["StnID"]
durations = ["1h", "24h"]
compute_ordinary_events_from_hpd(stations, durations)